# Linear Regression

In [1]:
import os 
import numpy as np
# import sys
# sys.path.append('..')

try:
    import matplotlib.pyplot as plt
    %matplotlib inline
except:
    print('matplotlib importation passed')

import tensorflow as tf
from tensorflow.python.layers import core as core_layers
try:
    from mpl_toolkits.mplot3d import Axes3D
except: 
    print('Axes3D importation passed')

In [2]:
def reset_graph(seed_=42):
    """
    Utility function to reset current tensorflow computation graph and set the random seed.
    """
    tf.reset_default_graph()
    tf.set_random_seed(seed_)
    np.random.seed(seed_)

help(reset_graph)

Help on function reset_graph in module __main__:

reset_graph(seed_=42)
    Utility function to reset current tensorflow computation graph and set the random seed.



## We use artificial data for the following two specifications of regression:

### Linear Regression

$ y(x) = a + b_1 \cdot X_1 + b_2 \cdot X_2 + b_3 \cdot X_3 + \sigma \cdot \varepsilon $ 

where $ \varepsilon \sim N(0, 1) $ is a Gaussian noise, and $ \sigma $ is its volatility, 
with the following choice of parameters:

$ a = 1.0 $

$ b_1, b_2, b_3 = (0.5, 0.2, 0.1) $

$ \sigma = 0.1 $

$ X_1, X_2, X_3 $ will be uniformally distributed in $ [-1,1] $

### Non-Linear Regression

$ y(x) = a + w_{00} \cdot X_1 + w_{01} \cdot X_2 + w_{02} \cdot X_3 + w_{10} \cdot X_1^2 + w_{11} \cdot X_2^2 + w_{12} \cdot X_3^2 +  \sigma \cdot \varepsilon $ 

where

$ w = [[1.0, 0.5, 0.2],[0.5, 0.3, 0.15]]  $

and the rest of parameters is as above, with the same values of $ X_i $

In [3]:
def generate_data(n_points_=10000, n_features_=3, use_nonlinear_=True, noise_std_=0.1, train_test_split_=4):
    """
    Parameters:
        n_points_ - the number of data points to generate
        n_features_ - a positive int 
        use_nonlinear_ - generate non-linear data if true
        train_test_split_ - a portion of data to test

    Return:
        X_train, Y_train, X_test, Y_test, n_train, n_features
    """
    if use_nonlinear_:
        weights = np.array([[1.0, 0.5, 0.2], [0.5, 0.3, 0.15]])
    else:
        weights = np.array([1.0, 0.5, 0.2])

    bias = np.ones(n_points_).reshape(-1, 1)  # 'a' - a vector of size (n_points_, ) 
    low = - np.ones( (n_points_, n_features_), 'float' )  # a vector of size (n_points_, ) 
    high = np.ones( (n_points_, n_features_), 'float' )  # an array of size (n_points_, n_features_) 

    np.random.seed(42)
    X = np.random.uniform( low=low, high=high )  # X_1, X_2, X_3 will be uniformally distributed in [-1,1]

    np.random.seed(42)
    noise = np.random.normal(size=(n_points_, 1))  # e is Gaussian noise
    # noise_std_ = 0.1  # ?? why again

    if use_nonlinear_:
        Y = bias + \
            np.dot(X, weights[0, :]).reshape(-1, 1) + \
            np.dot(X*X, weights[1, :]).reshape(-1, 1) + \
            noise_std_*noise
    else:
        Y = bias + \
            np.dot(X, weights).reshape(-1, 1) + \
            noise_std_*noise
    
    n_test = int(n_points_/train_test_split_)
    n_train = n_points_ - n_test
    
    X_train = X[:n_train,:]
    Y_train = Y[:n_train].reshape((-1,1))

    X_test = X[n_train:,:]
    Y_test = Y[n_train:].reshape((-1,1))
    
    return X_train, Y_train, X_test, Y_test, n_train, n_features_

X_train, Y_train, X_test, Y_test, n_train, n_features = generate_data(use_nonlinear_=False)
X_train.shape, Y_train.shape

((7500, 3), (7500, 1))

### 1. Linear Regression with Numpy

In [4]:
def numpy_lin_regress(X_train_, Y_train_):
    """
    Implements linear regression model using numpy module

    Parameters:
        X_train_ - np.array of size (n by k) where n is number of observations of independent variables and k is number of variables
        Y_train_ - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    Return:
        np.array of size (k+1 by 1) of regression coefficients - 변수에 대한 선형회귀의 계수
    """
    ### START CODE HERE ### (≈ 3 lines of code)
    number_of_features = len(X_train_[0])  # 7500 elements of size 3, n_train
    X = np.hstack( (np.ones( len(X_train_) ).reshape(-1, 1), X_train_ ) )  # # add the column of ones
    theta_numpy = np.linalg.inv( X.T.dot(X) ).dot(X.T).dot(Y_train_)
    ### END CODE HERE ###

    return theta_numpy

In [5]:
numpy_lin_regress(X_train, Y_train)

array([[0.99946227],
       [0.99579039],
       [0.499198  ],
       [0.20019798]])

### 2. Linear Regression with Sklearn

In [6]:
def sklearn_lin_regress(X_train_, Y_train_):
    """
    Parameters:
        X_train  - np.array of size (n by k) where n is number of observations of independent variables and k is number of variables
        Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    Return:
        np.array of size (k+1 by 1) of regression coefficients
    """ 
    from sklearn.linear_model import LinearRegression
    lin_reg = LinearRegression()
    ### START CODE HERE ### (≈ 3 lines of code)
    # use lin_reg to fit training data
    lin_reg.fit(X_train_, Y_train_)
    theta_sklearn = np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T]
    ### END CODE HERE ###

    return theta_sklearn

In [7]:
sklearn_lin_regress(X_train, Y_train)

array([[0.99946227],
       [0.99579039],
       [0.499198  ],
       [0.20019798]])

### Linear Regression with Tensorflow

In [8]:
def tf_lin_regress(X_train_, Y_train_):
    """
    Parameters:
        X_train - np.array of size (n by k) where n is number of observations of independent variables and k is number of variables
        Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    Return:
        np.array of size (k+1 by 1) of regression coefficients
    """
    ### START CODE HERE ### (≈ 7-8 lines of code)
    X_np = np.hstack((np.ones(len(X_train_)).reshape((-1, 1)), X_train_))  # add the column of ones
    
    # define theta for later evaluation
    X = tf.constant(X_np, dtype=tf.float32, name='X')
    Y = tf.constant(Y_train_, dtype=tf.float32, name='Y')
    XT = tf.transpose(X)

    theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), Y)

    ### END CODE HERE ###
    with tf.Session() as sess:
        theta_value = theta.eval()

    return theta_value

In [9]:
tf_lin_regress(X_train, Y_train)

array([[0.9994622 ],
       [0.9957907 ],
       [0.49919802],
       [0.20019804]], dtype=float32)

In [10]:
class LinRegressNormalEq:
    """
    Implements the normal equation, MLE
    """
    def __init__(self, n_features_, lr_=0.05, L_=0):
        import math as m
        self.X = tf.placeholder(tf.float32, [None, n_features_], name='X')
        self.Y = tf.placeholder(tf.float32, [None, 1], name='Y')
        self.theta_in = tf.placeholder(tf.float32, [n_features_+1, None])
        
        data_plus_bias = tf.concat([tf.ones([tf.shape(self.X)[0], 1]), self.X], axis=1)

        XT = tf.transpose(data_plus_bias)

        self.theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, data_plus_bias)), XT), self.Y)
        self.lr_mse = tf.reduce_mean(tf.square(tf.matmul(data_plus_bias, self.theta_in) - self.Y))

        self.weights = tf.Variable(tf.random_normal([n_features+2, 1]))
        self.output = tf.matmul(data_plus_bias, self.weights[:-1, :])

        gauss = tf.distributions.Normal(loc=0.0, scale=1.0)

        sigma = 0.0001 + tf.square(self.weights[-1])  # last model weight
        
        pi = tf.constant(m.pi)

        log_LL = tf.log(0.00001 + ( 1/( tf.sqrt(2*pi)*sigma)) * gauss.prob((self.Y - self.output) / sigma ) )  
        self.loss = - tf.reduce_mean(log_LL)

        self.train_step = (tf.train.AdamOptimizer(lr_).minimize(self.loss), -self.loss)

In [14]:
def run_normal_eq(X_train_, Y_train_, X_test_, Y_test_, lr_=0.05):
    """
    Implements normal equation using tf, trains the model using training set. Tests the model quality by computing MSE of the test data set.

    Parameter: 
        X_train_ - np.array of size (n, k)
        Y_train_ - np.array of size (n, 1)
        X_test_ - np.array of size (n, k)
        Y_test_ - np.array of size (n, 1)

    Return:
        - np.array of size (k+1, 1) of regression coefficients
        - MSE of the test data set
        - MSE of the train data set
    """
    n_features = X_train_.shape[1]
    model = LinRegressNormalEq(n_features_=n_features, lr_=lr_)

    # train the model

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # Normal equation for Linear Regression
        (_ , loss), weights = sess.run((model.train_step, model.weights), feed_dict={
                model.X: X_train_,
                model.Y: Y_train_
                })

        theta_value = model.theta
        lr_mse_train = model.lr_mse
        lr_mse_test = model.lr_mse

    return theta_value, lr_mse_train, lr_mse_test

theta_value, lr_mse_train, lr_mse_test = run_normal_eq(X_train, Y_train, X_test, Y_test)

In [12]:
run_normal_eq(X_train, Y_train, X_test, Y_test)

AttributeError: __enter__